<a href="https://colab.research.google.com/github/zizilnam/Mini_Project_EDA_Instacart_Kaggle/blob/main/instacart_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pip

In [ ]:
!pip install apyori


In [ ]:
!python --version

Python 3.7.11


In [ ]:
import os
import pandas as pd
import numpy as np
from apyori import apriori
from datetime import datetime
from itertools import combinations
import matplotlib.pyplot as plt

In [ ]:
os.chdir('/content/drive/MyDrive/KDT/instacart')

# Loading and studying the files:

In [ ]:
aisles = pd.read_csv('aisles.csv')
aisles.dtypes

FileNotFoundError: ignored

In [ ]:
aisles

Here we have the names of the runners and the primary key for their identification. Let's check now for misssing values.

In [ ]:
aisles.isna().sum(axis = 0)

As there are no missing files, no special treatment for this database will be necessary

In [ ]:

departments = pd.read_csv('/content/drive/MyDrive/KDT/instacart/departments.csv')
departments.dtypes

In [ ]:
departments

As with aisles, here we have two columns as the primary key and the name of the department,

In [ ]:
departments.isna().sum(axis = 0)

In [ ]:
products = pd.read_csv('/content/drive/MyDrive/KDT/instacart/products.csv')
products.dtypes

In [ ]:
products

We can see that have something new here, the appearance of foreign keys, the products are connected to the entrance and the corridor. It is possible to check the names of these corridors and departments:

In [ ]:
aisles[aisles['aisle_id'] == 61]

In [ ]:
departments[departments['department_id'] == 19]

In [ ]:
products.describe()

The "count" show equal values, indicating that there are no missing values.

In [ ]:
orders_path = 'orders.csv'
orders = pd.read_csv(orders_path)
orders.dtypes

In [ ]:
orders

In [ ]:
orders.shape

In [ ]:
orders.eval_set.value_counts()

In this column, there is already the separation of the database in training and testin. We will filter only the "prior"

In [ ]:
orders = orders[orders.eval_set == 'prior']

We will only use "prior" records

as this column does not represent any information, it is only the division of the database, we will exclude it:

In [ ]:
orders.drop('eval_set', axis = 1, inplace = True)

In [ ]:
orders.isna().sum(axis = 0)

Every user makes his first and is marked with NaN because hehad never made a purchase there before, so there is no value for days before this purchase, after all it was his first. Therefore, this NaN is not considered a missing value. We can locate where these first orders placed by users are located:

In [ ]:
orders.loc[orders.days_since_prior_order.isna()]

Let's see this with just 100 number of orders:

In [ ]:
plt.plot(orders.order_number[:100])
plt.title('Sequence of order number')
plt.xlabel('Sequence in the dataframe')
plt.ylabel('Order Number')

This graph shows us the number of orders made by 100 users, the first user had 10order numbers, right after that we see a low point that indicates the second user, this grows until it shows a number of orders of approximately 13 or 14. Within those 100 users, the one that reached 20 was placed in more orders. Let's view the distribution considering the day of the week and the hours of the day:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
# ax[0].boxplot(orders.order_dow[:-1])
ax[0].boxplot(orders[orders['order_dow'].notnull()].order_dow)
ax[0].set_title('Boxplot day of week')
ax[0].set_ylabel('day of week')
ax[1].hist(orders.order_hour_of_day)
ax[1].set_title('Histogram hour of day')
ax[1].set_xlabel('hour')
ax[1].set_ylabel('count')

The average purchase is indicated by the orange line, showing that it is day 3. We also can see we have some orders in the hour 0 but the most orders are in hours between 14 and 16. Finally an analysis on days_since_prior_order

In [ ]:
plt.figure(figsize = (15,5))
plt.bar(range(100), orders.days_since_prior_order[:100] + 1)
plt.title('Days since prior order')
plt.xlabel('index')
plt.ylabel('days since prior order + 1')

Here we see how many days later he returned after making the last purchase. Using the first user, we see that it considered NaN and counted 0 days but soon afterwards he said that it returned after 16/17days for a new purchase. The blanks mean th NaN.

In [ ]:
order_products = pd.read_csv('order_products__prior.csv')
print(order_products.dtypes)
order_products

In [ ]:
order_products

The order_id is related to the customer, the add_to_cart_order is the order to which the product is added to the cart, finally the reordered means if the product has already been ordered in previous purchases

In [ ]:
order_products.isna().sum(axis=0)

# Performing data exploration

In [ ]:
orders_apriori = orders.copy()
orders_user = orders.groupby('user_id')['order_number'].max()
orders_user

Let's create a data frame that returns not only the user_id and the maximum number of order_number but also the products purchased by the user.

In [ ]:
orders.head()

In [ ]:
products_user = orders[['order_id', 'user_id']].merge(order_products[['order_id', 'add_to_cart_order']].groupby('order_id').max().rename({'add_to_cart_order' : 'order_size'}, axis = 1),
                                                      on = 'order_id')
products_user